In [50]:
import sys
sys.path.append('..')
sys.path.append('../..')
import torch
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import math
import numpy as np
from PIL import Image
import copy
import logging
import time
import os
from utils.arg_extract import seed_everything
from old.Network import Network
device='cpu'

In [59]:
def initial_input(model, input_frames, batch_images, starting_point, num_input_frames, num_channels, device, training):
    """
    var           size
    batch_images  [16, 100, 128, 128]
    input_frames  [16, 5, 128, 128]
    output_frames  [16, 1, 128, 128]
    target_frames  [16, 1, 128, 128]
    """
    output_frames = model(input_frames.to(device), mode='initial_input', training=training)
    target_idx = starting_point + num_input_frames
    target_frames = batch_images[:, target_idx * num_channels:(target_idx + 1) * num_channels, :, :].to(device)
    return output_frames, target_frames

def reinsert(model, input_frames, output_frames, target_frames, batch_images, starting_point, num_input_frames, future_frame_idx, num_channels, device, training):
    output_frames = torch.cat((output_frames, model(input_frames, mode="reinsert", training=training)), dim=1).to(device)
    target_idx = starting_point + future_frame_idx + num_input_frames
    target_frames = torch.cat((target_frames, batch_images[:, target_idx*num_channels:(target_idx + 1) * num_channels, :, :].to(device)), dim=1)
    return output_frames, target_frames

def propagate(model, output_frames, target_frames, batch_images, starting_point, num_input_frames, future_frame_idx, num_channels, device, training):
    output_frames = torch.cat((output_frames, model(torch.Tensor([0]), mode="propagate", training=training)), dim=1).to(device)
    target_idx = starting_point + future_frame_idx + num_input_frames
    target_frames = torch.cat((target_frames, batch_images[:, target_idx* num_channels:(target_idx+ 1) * num_channels, :, :].to(device)), dim=1)
    return output_frames, target_frames

In [74]:
from utils.experiment import get_normalizer, create_new_datasets, create_dataloaders
normalizer = get_normalizer('none')
datasets = create_new_datasets('/Users/stathis/Code/thesis/wave_propagation/Video_Data/', normalizer)
dataloaders = create_dataloaders(datasets, 16, 1)
seed_everything(12345)
batch_images = next(iter(dataloaders['train']))

In [99]:
batch_images[0,0,20:25,20:25]

tensor([[0.0078, 0.0039, 0.0118, 0.0196, 0.0314],
        [0.0078, 0.0000, 0.0118, 0.0039, 0.0392],
        [0.0039, 0.0000, 0.0039, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0078, 0.0078, 0.0039, 0.0000, 0.0000]])

In [97]:
num_input_frames = 5
num_output_frames = 20
reinsert_frequency=10
num_channels = 1

tensor([[ 0.0616,  0.0485,  0.0629, -0.0050, -0.0533, -0.0411, -0.0169],
        [ 0.0086, -0.0207, -0.0368, -0.0052,  0.0403,  0.0531, -0.0315],
        [-0.0366, -0.0029,  0.0281,  0.0286, -0.0238,  0.0221, -0.0109],
        [-0.0077, -0.0518,  0.0458, -0.0451,  0.0159, -0.0320, -0.0539],
        [-0.0041,  0.0212, -0.0586,  0.0046, -0.0240, -0.0091, -0.0285],
        [-0.0080,  0.0575, -0.0614, -0.0302,  0.0543, -0.0034, -0.0133],
        [ 0.0215, -0.0270,  0.0262, -0.0394, -0.0192,  0.0489,  0.0063]],
       grad_fn=<SliceBackward>)

In [106]:
seed_everything(12345)
model = Network(device, 1)
optimizer_algorithm = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_algorithm, mode='min', factor=0.1, patience=7)
print(list(model.parameters())[0][0,0,:])

training = True
model.train()           # initialises training stage/functions
mean_loss = 0

starting_point = 0
model.reset_hidden(batch_size=batch_images.size()[0], training=True)

lr_scheduler.optimizer.zero_grad()
for future_frame_idx in range(num_output_frames):
    if future_frame_idx == 0:
        input_frames = batch_images[:, starting_point * num_channels:(starting_point + num_input_frames) * num_channels, :, :].clone()
        output_frames, target_frames = initial_input(model, input_frames, batch_images, starting_point, num_input_frames, num_channels, device, training) 
    elif future_frame_idx == reinsert_frequency:
        input_frames = output_frames[:, -num_input_frames * num_channels:, :, :].clone()
        output_frames, target_frames = reinsert(model, input_frames, output_frames, target_frames, batch_images, 
                                                starting_point, num_input_frames, future_frame_idx, num_channels, device, training)
    else:
        output_frames, target_frames = propagate(model, output_frames, target_frames, batch_images, starting_point, 
                                                 num_input_frames, future_frame_idx, num_channels, device, training)
#     print(output_frames.size())
loss = F.mse_loss(output_frames, target_frames)
print(loss.item())
loss.backward()
lr_scheduler.optimizer.step()
print(list(model.parameters())[0][0,0,:])

tensor([[ 0.0616,  0.0485,  0.0629, -0.0050, -0.0533, -0.0411, -0.0169],
        [ 0.0086, -0.0207, -0.0368, -0.0052,  0.0403,  0.0531, -0.0315],
        [-0.0366, -0.0029,  0.0281,  0.0286, -0.0238,  0.0221, -0.0109],
        [-0.0077, -0.0518,  0.0458, -0.0451,  0.0159, -0.0320, -0.0539],
        [-0.0041,  0.0212, -0.0586,  0.0046, -0.0240, -0.0091, -0.0285],
        [-0.0080,  0.0575, -0.0614, -0.0302,  0.0543, -0.0034, -0.0133],
        [ 0.0215, -0.0270,  0.0262, -0.0394, -0.0192,  0.0489,  0.0063]],
       grad_fn=<SliceBackward>)
1.5056633949279785
tensor([[ 0.0615,  0.0484,  0.0628, -0.0051, -0.0534, -0.0412, -0.0170],
        [ 0.0085, -0.0208, -0.0369, -0.0051,  0.0404,  0.0530, -0.0316],
        [-0.0367, -0.0028,  0.0282,  0.0287, -0.0237,  0.0220, -0.0110],
        [-0.0078, -0.0517,  0.0459, -0.0451,  0.0158, -0.0321, -0.0540],
        [-0.0040,  0.0213, -0.0585,  0.0045, -0.0241, -0.0092, -0.0286],
        [-0.0079,  0.0576, -0.0613, -0.0301,  0.0542, -0.0035, -0.0134],

In [111]:
seed_everything(12345)
from models.AR_LSTM import AR_LSTM
model2 = AR_LSTM(num_input_frames, num_output_frames, reinsert_frequency, device)
optimizer_algorithm2 = optim.Adam(filter(lambda p: p.requires_grad, model2.parameters()), lr=1e-4)
lr_scheduler2 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_algorithm2, mode='min', factor=0.1, patience=7)
print(list(model2.parameters())[0][0,0,:])
input_frames = batch_images[:, starting_point * num_channels:(starting_point + num_input_frames) * num_channels, :, :].clone()
output_frames = model2.get_future_frames(input_frames, num_output_frames)
loss2 = F.mse_loss(output_frames, target_frames)
print(loss2.item())
loss2.backward()
lr_scheduler2.optimizer.step()
print(list(model2.parameters())[0][0,0,:])

tensor([[ 0.0616,  0.0485,  0.0629, -0.0050, -0.0533, -0.0411, -0.0169],
        [ 0.0086, -0.0207, -0.0368, -0.0052,  0.0403,  0.0531, -0.0315],
        [-0.0366, -0.0029,  0.0281,  0.0286, -0.0238,  0.0221, -0.0109],
        [-0.0077, -0.0518,  0.0458, -0.0451,  0.0159, -0.0320, -0.0539],
        [-0.0041,  0.0212, -0.0586,  0.0046, -0.0240, -0.0091, -0.0285],
        [-0.0080,  0.0575, -0.0614, -0.0302,  0.0543, -0.0034, -0.0133],
        [ 0.0215, -0.0270,  0.0262, -0.0394, -0.0192,  0.0489,  0.0063]],
       grad_fn=<SliceBackward>)
1.5056633949279785
tensor([[ 0.0615,  0.0484,  0.0628, -0.0051, -0.0534, -0.0412, -0.0170],
        [ 0.0085, -0.0208, -0.0369, -0.0051,  0.0404,  0.0530, -0.0316],
        [-0.0367, -0.0028,  0.0282,  0.0287, -0.0237,  0.0220, -0.0110],
        [-0.0078, -0.0517,  0.0459, -0.0451,  0.0158, -0.0321, -0.0540],
        [-0.0040,  0.0213, -0.0585,  0.0045, -0.0241, -0.0092, -0.0286],
        [-0.0079,  0.0576, -0.0613, -0.0301,  0.0542, -0.0035, -0.0134],